In [1]:
import lib.xmlreader as xml
import re
import pandas as pd
import os
import gensim
import sys
import numpy as np
import pickle
from lib.utils import tokenize
from collections import Counter

DATA_PATH = '../database/TASS/'

In [2]:
train_docs = xml.readXML(DATA_PATH + "TASS2017/task1-Training.xml")
test_docs  = xml.readXML(DATA_PATH + "TASS2017/task1-Development.xml" )

In [3]:
trn_text   = [t.content  for t in train_docs]
trn_label  = [t.polarity for t in train_docs]

test_text   = [t.content  for t in test_docs]
test_label  = [t.polarity for t in test_docs]

In [4]:
col_names = ['labels','text']

In [5]:
df_train = pd.DataFrame({'text':trn_text, 'labels':trn_label}, columns=col_names)
df_test  = pd.DataFrame({'text':test_text, 'labels':test_label}, columns=col_names)

In [6]:
#CSV_PATH = DATA_PATH + 'csv_data/'

#if not os.path.exists(CSV_PATH):
#    os.makedirs(CSV_PATH)

In [7]:
#df_train.to_csv(DATA_PATH + 'csv_data/train.csv', index=False)
#df_test.to_csv(DATA_PATH  + 'csv_data/test.csv',  index=False)

In [8]:
#df_train = pd.read_csv(DATA_PATH + 'csv_data/train.csv')
#df_test = pd.read_csv(DATA_PATH + 'csv_data/test.csv')

In [9]:
df_train.head()

,labels,text
0,0,@myendlesshazza a. que puto mal escribo\n\nb. ...
1,0,@estherct209 jajajaja la tuya y la d mucha gen...
2,1,Quiero mogollón a @AlbaBenito99 pero sobretodo...
3,0,Vale he visto la tia bebiendose su regla y me ...
4,1,@Yulian_Poe @guillermoterry1 Ah. mucho más por...


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 2 columns):
labels    736 non-null int64
text      736 non-null object
dtypes: int64(1), object(1)
memory usage: 11.6+ KB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 2 columns):
labels    375 non-null int64
text      375 non-null object
dtypes: int64(1), object(1)
memory usage: 5.9+ KB


In [12]:
tokenize(df_train['text'][0], None)

{'original': '@myendlesshazza a. que puto mal escribo\n\nb. me sigo surrando help \n\n3. ha quedado raro el "cómetelo" ahí JAJAJAJA',
 'words': ['a',
  'que',
  'puto',
  'mal',
  'escribo',
  'b',
  'me',
  'sigo',
  'surrando',
  'help',
  '3',
  'ha',
  'quedado',
  'raro',
  'el',
  'cometelo',
  'ahi',
  'jajajaj'],
 'ratio': 0.0963855421686747,
 'clean': ' a. que puto mal escribo b. me sigo surrando help 3. ha quedado raro el cometelo ahi jajajaj',
 'class': None}

In [13]:
len(df_train['text'])

736

In [14]:
LOF_train = []
LOF_test  = []

for i in range(len(df_train['text'])):
    LOF_train.append(tokenize(df_train['text'][i], None)['words'])
    
for i in range(len(df_test['text'])):
    LOF_test.append(tokenize(df_test['text'][i], None)['words'])

In [15]:
print("Size of List Of Words (Train) : ", len(LOF_train))
print("Size of List Of Words (Test)  : ", len(LOF_test))

Size of List Of Words (Train) :  736
Size of List Of Words (Test)  :  375


In [16]:
LOF_test[0]

['1000',
 '10',
 'de',
 'verdad',
 'a',
 'ti',
 'que',
 'voy',
 'a',
 'decir',
 'petarda',
 'que',
 'te',
 'quiero',
 'mas',
 'que',
 'a',
 'mi',
 'mismo']

In [17]:
cnt = Counter(word for doc in LOF_train + LOF_test for word in doc )

In [18]:
len(cnt)

4344

In [19]:
cnt.most_common()[:10]

[('que', 637),
 ('de', 572),
 ('y', 431),
 ('no', 397),
 ('a', 360),
 ('me', 339),
 ('la', 338),
 ('en', 303),
 ('el', 300),
 ('es', 276)]

In [21]:
import collections

max_vocab = 4000
min_freq = 1
#max_vocab = len(cnt)
#max_vocab = 4000
#min_freq  = 0


itos = [o for o,c in cnt.most_common(max_vocab) if c > min_freq]

itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

1243

In [22]:
len(stoi)

1243

In [23]:
train_lm = np.array([[stoi[o] for o in p] for p in LOF_train])
test_lm  = np.array([[stoi[o] for o in p] for p in LOF_test])

In [24]:
SAVE_PATH = '../database/ulmfit/'
np.save(SAVE_PATH + 'tmp/train_labels.npy',trn_label)
np.save(SAVE_PATH + 'tmp/test_labels.npy',test_label)
#np.savez(DATA_PATH + 'tmp/train_labels.npz', trn_label=trn_label)
#np.savez(DATA_PATH + 'tmp/test_labels.npz', test_label=test_label)

In [25]:
np.save(SAVE_PATH + 'tmp/train_lm.npy',train_lm)
np.save(SAVE_PATH + 'tmp/test_lm.npy',test_lm)
#np.savez(DATA_PATH + 'tmp/train_lm.npz', train_lm=train_lm)
#np.savez(DATA_PATH + 'tmp/test_lm.npz', test_lm=test_lm)
pickle.dump(itos, open(SAVE_PATH + 'tmp/itos.pkl', 'wb'))

In [26]:
type(train_lm[0][0])

int

In [27]:
type(train_lm[0][0])

int

In [28]:
max(np.concatenate(train_lm))

1175

In [29]:
max(np.concatenate(test_lm))

1242

In [30]:
itos[-1]

'antiguo'

In [31]:
itos[:10]

['_unk_', '_pad_', 'que', 'de', 'y', 'no', 'a', 'me', 'la', 'en']

In [32]:
test_label[:10]

[1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

In [33]:
trn_label[:10]

[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]